In [ ]:
# 1

import secrets

def random_pad(data: bytes, block_size: int) -> bytes:
    pad_size = block_size - len(data) % block_size
    if pad_size == block_size and len(data) > 0:
        pad_size = 0
    return data + secrets.token_bytes(pad_size)

assert len(random_pad(b'', 8)) == 8
assert len(random_pad(b'1', 8)) == 8
assert len(random_pad(b'11111111', 8)) == 8
assert len(random_pad(b'111111111', 8)) == 16

In [ ]:
# 2

def counter_pad(data: bytes, block_size: int) -> bytes:
    pad_size = block_size - len(data) % block_size
    
    data = bytearray(data)
    for i in range(pad_size):
        data.append(i & 0xff)
    return bytes(data)
        
def counter_unpad(data: bytes, block_size: int) -> bytes:
    pad_size = data[-1] + 1
    return data[:-pad_size]


import secrets
assert counter_pad(b'aabb', 8) == (b'aabb\x00\x01\x02\x03')
assert counter_pad(b'aabb', 4) == (b'aabb\x00\x01\x02\x03')
assert len(counter_pad(b'aabbccdd', 8)) == 16
assert counter_pad(b'', 8) == (b'\x00\x01\x02\x03\x04\x05\x06\x07')
assert counter_unpad(counter_pad(b'aabb', 8), 8) == b'aabb'
assert all([ len(counter_unpad(counter_pad(secrets.token_bytes(i), 8), 8)) == i for i in range(100) ])

In [29]:
# 3: PKCS#7 padding

def pad(data: bytes, block_size: int) -> bytes | None:
    if block_size < 1 or block_size > 256:
        return None
    pad_len = block_size - (len(data) % block_size)
    return data + bytes([pad_len] * pad_len)

def unpad(data: bytes) -> bytes | None:
    pad_len = data[-1]
    if pad_len < 1 or pad_len > len(data):
        return None
    return data[:-pad_len]


assert pad(b'hello', 8) == b'hello\x03\x03\x03'
assert pad(b'welcome stranger', 8) == b'welcome stranger\x08\x08\x08\x08\x08\x08\x08\x08'
assert pad(b'welcome stranger!', 8) == b'welcome stranger!\x07\x07\x07\x07\x07\x07\x07'
assert pad(b'', 4) == b'\x04\x04\x04\x04'
assert unpad(b'\x02\x02\x02\x02') == b'\x02\x02'
assert unpad(pad(b'hello', 8)) == b'hello'
assert unpad(pad(b'this is a long message, taking up multiple blocks', 4)) == b'this is a long message, taking up multiple blocks'

In [3]:
# 4: Generate a 4 byte long message that has the same hash as the message "hello"

def hash(data: bytes) -> int:
    result = 0xFF
    for byte in data:
        result ^= byte
    return result

def find_collision(target: bytes) -> bytes:
    for i in range(256):
        for j in range(256):
            for k in range(256):
                for l in range(256):
                    candidate = bytes([i, j, k, l])
                    if hash(candidate) == hash(target):
                        return candidate
    return b''

print(b'hello!')
print(hash(b'hello!'))
print(find_collision(b'hello!'))

b'hello!'
188
b'\x00\x00\x00C'


In [4]:
#5: Generate any message in md5 where the first 3 bytes are all 0

import hashlib

def find_md5_by_prefix(prefix: bytes) -> tuple[bytes, bytes]:
    counter = 0
    while True:
        candidate = counter.to_bytes(16, 'big').lstrip(b'\x00')
        digest = hashlib.md5(candidate).digest()
        if digest.startswith(prefix):
            return (candidate, digest)
        counter += 1

assert find_md5_by_prefix(b'\x00')[1].startswith(b'\x00')
assert find_md5_by_prefix(b'\x00\x00')[1].startswith(b'\x00\x00')
assert find_md5_by_prefix(b'\xff\xff')[1].startswith(b'\xff\xff')
assert find_md5_by_prefix(b'\x00\x00\x00')[1].startswith(b'\x00\x00\x00')

In [2]:
import itertools
def generate_bytes():
    length = 1
    while True:
        for message in itertools.product(range(256), repeat=length):
            yield bytes(message)
        length += 1

iterator = generate_bytes()
for i in range(20):
    print(next(iterator))

b'\x00'
b'\x01'
b'\x02'
b'\x03'
b'\x04'
b'\x05'
b'\x06'
b'\x07'
b'\x08'
b'\t'
b'\n'
b'\x0b'
b'\x0c'
b'\r'
b'\x0e'
b'\x0f'
b'\x10'
b'\x11'
b'\x12'
b'\x13'
